In [1]:
import pandas as pd
from data.data import Data
from sklearn.linear_model import LinearRegression

ROWS = 200 # Limit 6000

data_raw = Data().get_all_data()


# 8670 ROWS INPUT #
df1 = data_raw['twitter_MBTI']
df1 = df1[['label', 'text']]
df1.rename(columns={'label': 'type'}, inplace=True)
df1 = df1.sample(n = ROWS, ignore_index = True)

# 106062 ROWS INPUT #
df2 = data_raw['MBTI 500']
df2 = df2[['type', 'posts']]
df2.rename(columns={'posts': 'text'}, inplace=True)
df2 = df2.sample(n = ROWS, ignore_index = True) # > 15000 combined, > 20000 df1 + df2

## SPECIFCALLY CREATING DIV 0 ERROR, ASK ABOUT IT ##
# 7811 ROWS INPUT #
df3 = data_raw['mbti_1']
df3.rename(columns={'posts': 'text'}, inplace=True)
df3 = df3.iloc[:1000]
# df3 = df3.sample(n = ROWS, ignore_index = True)
# ####################################################

## Combined all data for PP if desired
data_combined = pd.concat([df1, df2, df3], ignore_index=True)
data_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    1400 non-null   object
 1   text    1400 non-null   object
dtypes: object(2)
memory usage: 22.0+ KB


/var/folders/zm/96y7vfh93l90fb_pxv9n_ndm0000gn/T/ipykernel_98596/1737909153.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'label': 'type'}, inplace=True)


In [4]:
df = data_raw['MBTI 500']
df = df[['type', 'posts']]
df.rename(columns={'posts': 'text'}, inplace=True)
df = df.sample(n = 10, ignore_index = True)

estimation = df['type'].iloc[:1].tolist()[0]
df_pred = df.iloc[:1].drop(columns =['type'])

estimation

'INFJ'

In [5]:
from scripts.Preprocessing_full import prediction_preprocessing, prediction_vectorize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

df_pred_pp = prediction_preprocessing(df_pred)
df_pred_list = prediction_vectorize(df_pred_pp)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ericferole/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ericferole/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ericferole/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: 'ei_vectorizer.pkl'

In [10]:
predict_model(texts = df_pred_list, verbose = False)
print(f"Actual = {estimation}")

Final Prediction = ESTP
Actual = ENTJ
